### An example of 2D CNN on MNIST data (numpy implementation)
Wei Li

In [1]:
import os
import sys

sys.path
# If running the .py script, uncomment the next line
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))
# add parent directory: adds the parent directory of the module requiring it (__file__)
# to the beginning of the module search path.

# change the working directory to the parent folder
os.chdir("..")

import matplotlib.pyplot as plt
from utils import get_data_utils
from utils import data_processor
from models.cnn import *
from nn.modules.loss import *
from nn.modules.activation import *
from nn.modules.linear import *
from nn.modules.initializer import *
from optim.sgd import *
from optim.adam import *
from evaluation.multiclass_eval import *
from torchvision import transforms

import numpy as np
import random

random_seed = 123
os.environ["PL_GLOBAL_SEED"] = str(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

In [2]:
# %pip install watermark
%load_ext watermark
%watermark -a "Wei Li" -u -t -d -v -p numpy,torch,torchvision

Author: Wei Li

Last updated: 2023-12-11 22:47:20

Python implementation: CPython
Python version       : 3.8.17
IPython version      : 8.12.2

numpy      : 1.21.5
torch      : 1.12.1
torchvision: 0.13.1



In [3]:
##########################
### MNIST DATASET (Numpy data)
##########################

transformCompose = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

# obtain MNIST numpy files
dataset = get_data_utils.get_np_mnist(
    validation_fraction=0.2,
    train_transforms=transformCompose,
    test_transforms=transformCompose,
)

train_x = dataset[0]  # shape (size , 1, 32, 32), value in [-1, 1]
train_y = dataset[1]  # shape (size , 1)
valid_x = dataset[2]
valid_y = dataset[3]
test_x = dataset[4]
test_y = dataset[5]


# process data targets
train_y = data_processor.to_onehot(train_y, 10)
valid_y = data_processor.to_onehot(valid_y, 10)
test_y = data_processor.to_onehot(test_y, 10)

train_x.dtype, train_x.shape  # (48000, 1, 32, 32)
train_y.dtype, train_y.shape  # (48000, 10)


# train, validation, test data
train_data = [train_x, train_y]
valid_data = [valid_x, valid_y]
test_data = [test_x, test_y]

In [4]:
print("shape, min, max, mean and std: train_x")
print(train_x.shape, np.min(train_x), np.max(train_x), np.mean(train_x), np.std(train_x))
print("min, max, mean and std: test_x")
print(test_x.shape, np.min(test_x), np.max(test_x), np.mean(test_x), np.std(test_x))

shape, min, max, mean and std: train_x
(48000, 1, 32, 32) -1.0 1.0 -0.73777825 0.5792735
min, max, mean and std: test_x
(10000, 1, 32, 32) -1.0 1.0 -0.7345314 0.5838259


In [5]:
########################
##### Lenet5  ##########
########################

# Lenet 5 (2D CNN)

# x (batch_size, in_channels=1, in_height=32, in_width=32)
# -> Conv2d (out_channels 6, kernel 5, stride 1)-> tanh 
# -> maxpool (kernel 2)
# -> Conv2d (out_channels 16, kernel 5, stride 1)-> tanh 
# -> maxpool (kernel 2)
# -> Flatten2D
# -> Linear (in_features=16*5*5, out_features=120) -> tanh
# -> Linear (in_features=120, out_features=84) -> tanh
# -> Linear (in_features=84, out_features=10) (-->identity)

# ---- set up the user-defined model  ---- #
input_dims = list(train_x.shape[2:4])
num_input_channels = train_x.shape[1]

out_channels = [6, 16]
kernel_sizes = [5, 5]
strides = [1, 1]
pool_kernel_sizes = [2, 2]
conv_activations = [Tanh(), Tanh()]
num_linear_neurons = [120, 84, 10]
linear_activations = [Tanh(), Tanh(), Identity()]
# The last activation is the activation that produces output
# we use identity because the CrossEntropyLoss() we use here
# is taking logits

conv_weight_init_fn = weight_init_He_CNN
linear_weight_init_fn = weight_init_He
bias_init_fn = bias_init_zeros
criterion = CrossEntropyLoss()
lr = 1e-1


lenet5_model = Lenet5(
    input_dims,
    num_input_channels,
    out_channels,
    kernel_sizes,
    strides,
    num_linear_neurons,
    conv_activations,
    linear_activations,
    conv_weight_init_fn,
    bias_init_fn,
    linear_weight_init_fn,
    pool_kernel_sizes,
    pool_mode="max",
)

type(lenet5_model)

# check some utility functions
lenet5_model.print_structure()

#print_dict(lenet5_model.layers_dict)
#print_keys(lenet5_model.paras_dict)

model_paras_list = lenet5_model.get_parameters()

-----------------------
The model architecture:
layer0:
	sublayer0: <nn.modules.conv.Conv2D object at 0x15aa1f6d0>
	sublayer1: <nn.modules.activation.Tanh object at 0x15aa1fb50>
layer1:
	sublayer0: <nn.modules.conv.Pool2D object at 0x15aa1f880>
layer2:
	sublayer0: <nn.modules.conv.Conv2D object at 0x15aa1f9a0>
	sublayer1: <nn.modules.activation.Tanh object at 0x15aa1fa90>
layer3:
	sublayer0: <nn.modules.conv.Pool2D object at 0x15a8b2730>
layer4:
	sublayer0: <nn.modules.conv.Flatten2D object at 0x15aa25910>
layer5:
	sublayer0: <nn.modules.linear.Linear object at 0x15aa25940>
	sublayer1: <nn.modules.activation.Tanh object at 0x15aa1fac0>
layer6:
	sublayer0: <nn.modules.linear.Linear object at 0x15aa257f0>
	sublayer1: <nn.modules.activation.Tanh object at 0x15aa1fc10>
layer7:
	sublayer0: <nn.modules.linear.Linear object at 0x15a8ef6a0>
	sublayer1: <nn.modules.activation.Identity object at 0x15aa1fe80>

---------------------------------
layers with learnable parameters:
layer0 
 (0)conv1d


In [6]:
# training
optimizer = SgdDecay(model_paras_list, lr=lr)
optimizers = [optimizer]

num_epochs = 15
batch_size = 512

output = trainer_multiclass(
    lenet5_model,
    optimizers,
    criterion,
    train_data,
    valid_data,
    num_epochs,
    batch_size,
    print_all=True,
)

training_losses, training_errors, validation_losses, validation_errors = output

# benchmark 
# Epoch: 015/015 | Train loss: 0.0995 | Validation loss: 0.1138 
# Epoch: 015/015 | Train error: 0.0306 | Validation error: 0.0348 

# this implementation has achieved virtually same level of accuracy as the Pytorch implememntation

Epoch: 001/015 | Batch 000~511/48000 | Train loss: 2.3230
Epoch: 001/015 | Batch 512~1023/48000 | Train loss: 2.7377
Epoch: 001/015 | Batch 1024~1535/48000 | Train loss: 2.5007
Epoch: 001/015 | Batch 1536~2047/48000 | Train loss: 2.2399
Epoch: 001/015 | Batch 2048~2559/48000 | Train loss: 2.1752
Epoch: 001/015 | Batch 2560~3071/48000 | Train loss: 1.9410
Epoch: 001/015 | Batch 3072~3583/48000 | Train loss: 1.8285
Epoch: 001/015 | Batch 3584~4095/48000 | Train loss: 1.7994
Epoch: 001/015 | Batch 4096~4607/48000 | Train loss: 1.6368
Epoch: 001/015 | Batch 4608~5119/48000 | Train loss: 1.7257
Epoch: 001/015 | Batch 5120~5631/48000 | Train loss: 1.5118
Epoch: 001/015 | Batch 5632~6143/48000 | Train loss: 1.4610
Epoch: 001/015 | Batch 6144~6655/48000 | Train loss: 1.2996
Epoch: 001/015 | Batch 6656~7167/48000 | Train loss: 1.2094
Epoch: 001/015 | Batch 7168~7679/48000 | Train loss: 1.1296
Epoch: 001/015 | Batch 7680~8191/48000 | Train loss: 1.1317
Epoch: 001/015 | Batch 8192~8703/48000 | Tr